In [ ]:
#JLL edit: add cell to check we are in the right jupyter kernel for our virtual environment
import sys
print(sys.executable)

<font size="5"><center> <b>Sandpyper: sandy beaches SfM-UAV analysis tools</b></center></font>

![im](images/banner.png)


<font face="Calibri">

# Profile extraction, LoD and data cleaning

<font size="3"> <b> Nicolas Pucino; PhD Student @ Deakin University, Australia </b> <br>

<b>This notebook covers the following concepts:</b>

- The ProfileSet class.
- Data extraction.
- Limit of Detections (LoDs).
</font>


</font>

In [ ]:
%matplotlib notebook

from pathlib import Path
import matplotlib.pyplot as plt
import pandas as pd
import rasterio as ras
from rasterio.plot import show

from sandpyper.sandpyper import ProfileSet
from sandpyper.common import get_sil_location, get_opt_k

pd.options.mode.chained_assignment = None  # default='warn'

## Introduction

The __ProfileSet__ class is an object that contains all the information necessary to carry a typical *sandpyper* analysis from start to end. Once instantiated, it will contains a number of global variables that store some fundamental monitoring charasteristics such as:

* paths to the data directories
* coordinate reference systems of each site
* location codes and search keywords
* some monitoring parameters such as alongshore transect spacing, acrosshore sampling step and cleaning steps used
* and of course, the elevation and colour profile data.

Moreover, a few key methods will:

1. extract data from the provided transects
2. cluster extracted points using unsupervised clustering algorithm KMeans
3. clean the dataset with user-provided watermasks, shoremasks and fine-tuning polygons.

Moreover, when the __ProfileSet__ class is instantiated, it prints out the number of DSMs and orthophotos found for each location in the provided folders and it also creates a dataframe storing filenames, CRSs, location codes and raw dates extracted from the filenames. This dataframe is useful as a sanity check to see if all the information are correct before proceeding with the actual profile extraction. It will be stored as the attribute *ProfileSet.check*.

Let's see here how it all works...

First, create the required monitoring global settings.

In [ ]:
# JLL edit: changed to my path to the folder

# set the path to the test data folder
test_data_folder = r"/Users/joshualeonlovell/Desktop/Work/LUH_HiWi_LuFi_JT/coauthor/sandpyper_fork/examples/test_data"

In [ ]:
# JLL edits: changed to mac compatible paths, added spaces for readability

# the paths to the DSM, orthophotos and transect directories
dirNameDSM   = Path(test_data_folder + r"/dsm_1m")
dirNameOrtho = Path(test_data_folder + r"/orthos_1m")
dirNameTrans = Path(test_data_folder + r"/transects")

# path to the LoD transects

lod_mode = Path(test_data_folder + r"/lod_transects")


# the location codes used for the monitored locations
loc_codes = ["mar","leo"]


# the keyword search dictionary
loc_search_dict = {   'leo': ['St','Leonards','leonards','leo'],
                   'mar': ['Marengo','marengo','mar'] }


# the EPSG codes of the coordinate reference systems for each location code (location) given in CRS string format
crs_dict_string = {
                  'mar': {'init': 'epsg:32754'},
                  'leo':{'init': 'epsg:32755'}
                   }

# the transect spacing of the transects
transects_spacing = 20

Then, create an instance of the ProfileSet class. Let's set 'check' parameter to 'all' to create the check dataframe
and store it in the variable P.check.


In [ ]:
# JLL edit: added spaces for readability

P = ProfileSet(dirNameDSM=dirNameDSM,
               dirNameOrtho=dirNameOrtho,
               dirNameTrans=dirNameTrans,
               transects_spacing=transects_spacing,
               loc_codes=loc_codes,
               loc_search_dict=loc_search_dict,
               crs_dict_string=crs_dict_string,
               check="all")

Check the infromation extracted from the formatted filenames.
All the CRSs in a line (survey) must match!

## Extraction of profiles from folder

After checking the check dataframe and all looks right, we are ready to actually extract the data from both DSMs and orhtophotos using the *.extract_profiles* method.
Importantly, we also extract the data for the Limit of Detections (LoD) computation, using some transects digitised over areas we expect __NOT TO BE CHANGING__ in elevation during the monitoring period, also known as calibration zones.

Thus, the most important parameters to set are:

1. __sampling step__: indicates the __cross-shore sampling distance (m)__ that we want to use along our transects. Beware, although you could use a very small sampling distance (UAV datasets tend to be between few to 10 cm pixel size), file dimension will increase significantly!.

2. __lod__: this can be the path to a directory containing the lod transects to use as lod, a number to use as lod for all surveys in all locations or None.

The parameter __mode__ is here set to 'all' as we want to extract pixel values from both the DSMs and orthophotos, while the parameter __tr_ids__ specifies the name of the field in the transect file is used to store the transect IDs.

In [ ]:
# JLL edit: added spaces for readability

# run extraction from DSMs and orthos with 1m sampling steps and add X and Y fields to output geodataframe.
# use LoDs profiles provided.

P.extract_profiles(mode='all', tr_ids='tr_id', sampling_step=1, add_xy=True, lod_mode=lod_mode)

__Note about missing values (numpy.nan)__

>NaNs might come from two different cases:
>1. extraction of points generated on transects falling __outside__ of the underlying raster extent
>2. points sampled from transect __inside__ the raster extent but containing NoData cells.
>
>Conveniently, the extraction profile function makes sure that if points fall outside the raster extent (case 1), those elevations are assigned a default nan value, in the NumPy np.nan form.
>In case 2, however, the values extracted depends on the definition of NaNs of the source raster format. In case the rasters provided do not have an assigned nan value in their metadata, the paramter __default_nan_values__ can be used to specify one.

Done!

Now the profile attribute of the ProfileSet object we are working with (here, *P*) stores the geopandas.GeoDataFrame containing the extracted data.

In [ ]:
# JLL edit: added and removed spaces for readability, adapted path for Mac (forward slashes), added %matplotlib inline
%matplotlib inline

f, ax = plt.subplots(figsize=(10,8), squeeze=True)
ortho_path = Path(test_data_folder + r"/orthos_1m/leo_20180606_ortho_resampled_1m.tif")

with ras.open(ortho_path,'r') as ortho:
    show(ortho, ax=ax)

P.profiles.query("location=='leo' and raw_date==20180606").plot(ax=ax, column='z', cmap='RdBu_r');

As you can see, the profiles include points extracted over areas that are not beach sediment, such as water, swash (inaccurate modeled elevation), vegetation or anything else that can be found aôn or around a beach that is not sand.

Computing mutitemporal elevation changes from a dataset of this kind would lead in very erroneous computation as it would include all the aforementioned non-sand points. Therefore, we need to clip out the water from each dataset, retain only a small area of interest landward that the principal foredune (if any) and classify each extracted point as sand or no-sand.

Let's see how we can achieve this using sandpyper and Qgis.

## Unsupervised clustering and labelling
( for method information, please check the documentation, [here](https://npucino.github.io/sandpyper/) )

### The challenge
In order to help discriminate and deal with non-sand points in our dataset, we can use Machine Learning (ML) to classify which points are sand and which ones are not.

Currently, sandpyper only facilitates the use of KMeans unsupervised clustering algorithm to accomplish this task. KMeans assigns points sharing similarities in the attribute space to one of a predefined number of clusters and assign these clusters one label. After the clustering is done, we, the human operator, need to visually look at the clustering results in a GIS, overlay the points on the orthophoto, visualise their color-coded labels and 'attach a meaning' to those clusters, which can be quite difficult as ML sees quite differently from us.

Of course one could also be tempted to use supervised machine learning techniques, however, considering that the cameras mounted on UAVs are not calibrated and the RGB images are not corrected for illumination variations, the training process would require a very big training dataset which must capture as much as variance in the classes of interest as possible.
In this case, the unsupervised procedure facilitated by sandpyper could help in constructing an high quality training dataset.

### Iterative Silhouette analysis with inflexion search
One of the most important parameter we must set before running KMeans is how many clusters we want to partition our points in. Considering that the next step is our visual checking of each cluster, we should choose a number that is not too high, as it would require too much time to visually assign a class to each cluster, but neighter too low, as this would dramatically increase the likelyhood of finding mixed clusters containing both points that are sand and no-sand.

In order to let the data guide our choice of number of clusters to use, sandpyper implements an automated iterative Silhouette analysis with inflexion search algorithm.
This algorithms use Silhouette coefficients to compute the clustering performances and allows to return a sub-optimal number of clusters to use in each survey with KMeans which minimise the likelyhood of having mixed clusters while also retaining a low number of clusters.

Sandpyper implements this in 3 steps:

1. __get_sil_location__: iteratively perform KMeans clustering and Silhouette Analysis with increasing number of clusters (k, specified in the `ks` parameter) for every survey, using the feature set specified in the parameter `feature_set`
2. __get_opt_k__ : uses the dataframe with Average Silhouette scores with different k for all surveys created by __get_sil_location__ function to find a sub-optimal number of clusters for each survey
3. __kmeans_sa__ method: to run KMeans algorithm with the specified sub-optimal number of clusters (k)

In [ ]:
# JLL edits: added spaces for readability, corrected grammar in comments, added question to comment I don't understand

# Run iteratively KMeans + SA using the feature_set provided
#feel free to add (JLL: add what?)

feature_set = ["band1","band2","band3","distance"]
sil_df = get_sil_location(P.profiles,
                          ks=(2,15), 
                          feature_set=feature_set,
                          random_state=10)

Find sub-optimal k by searching inflexion points where an additional cluster do not considerably degrade the overall clustering performance.

In [ ]:
# JLL edit: added and removed spaces for readability

opt_k = get_opt_k(sil_df, sigma=0)
opt_k

If we are not satisfied with the sub-optimal k returned by the algorithm, we can manually specify each survey k
by defining a dictionary.

Based on our observations on a dataset comprising 87 surveys, __10 clusters (k=10)__ is generally a good tradeoff, and set 

```python
opt_k={'leo_20180606': 10,
 'leo_20180713': 10,
 'leo_20180920': 10,
 'leo_20190211': 10,
 'leo_20190328': 10,
 'leo_20190731': 10,
 'mar_20180601': 10,
 'mar_20180621': 10,
 'mar_20180727': 10,
 'mar_20180925': 10,
 'mar_20181113': 10,
 'mar_20181211': 10,
 'mar_20190205': 10,
 'mar_20190313': 10,
 'mar_20190516': 10}
```
or, we could update one value only. For instance, we could change __mar_2019-05-16 dataset__ to 10 clusters simply by updating the existing `òpt_k` dictionary:
```python
opt_k={'mar_20190516': 10}
```

However, for the purpose of this test workflow, __please keep the original opt_k dictionary that Sandpyper computed__.

In [ ]:
# JLL edit: added spaces for readability

feature_set = ["band1","band2","band3","distance"]

P.kmeans_sa(opt_k, feature_set)

DONE! Now, the profile dataframe attribute has an additional column named __label_k__, which is the result of the survey specific KMeans algorithm.

We can now export the profile dataset as CSV in order to the open it as a comma-delimited file in your favourite GIS.
Use this command to save it to your favourite path:

```python
P.profiles.to_csv("your/favourite/location/profiles.csv")
```

Now that we have labelled the data points and exported as a CSV file, we need to:

1. import it into your favourite GIS and use the __coordinates__ column as geometry
2. using the filtering options, visualise only one survey at a time and assign one color per unique label_k value
3. overlay these points on the orthophoto of the survey they were extracted from
4. take notes of which label_k number is what class (sand, no_sand at least)
5. create the label corrections file (polygons), following the specifications descirbed in the previous notebook.

## Sand labeling and cleaning

Here below I provided the classes dictionaries of the test data clustering results.

Each class has its own dictionary. The keys store the survey identifier tag ('LocCode_yyyymmdd') and the values are lists of label_k associated with the dictionary class.

For instance in this example:

```python
sand_dict = {'leo_20180606':[5],
            'leo_20180713':[1,3,4],
            'mar_20180920':[]}
            
water_dict = {'leo_20180606':[4],
            'leo_20180713':[2,6],
            'mar_20180920':[1,2,3,4]}
```

In the St. Leonards survey of the 13th July 2018, the label_k 1,3 and 4 are sand, while the label_k 2 and 6 are water.
In Marengo, the 20th September 2018, no label_k represents sand while 1,2,3 and 4 are water.

Here below are reported the label dictionaries of the test data.

In [ ]:
# JLL edit: added spaces for readability

water_dict = {'leo_20180606':[0,9,10],
              'leo_20180713':[0,3,4,7],
              'leo_20180920':[0,2,6,7],
              'leo_20190211':[0,2,5],
              'leo_20190328':[2,4,5],
              'leo_20190731':[0,2,8,6],
              'mar_20180601':[1,6],
              'mar_20180621':[4,6],
              'mar_20180727':[0,5,9,10],
              'mar_20180925':[6],
              'mar_20181113':[1],
              'mar_20181211':[4],
              'mar_20190205':[],
              'mar_20190313':[],
              'mar_20190516':[4,7]}

no_sand_dict = {'leo_20180606':[5],
                'leo_20180713':[],
                'leo_20180920':[],
                'leo_20190211':[1],
                'leo_20190328':[],
                'leo_20190731':[1],
                'mar_20180601':[4,5],
                'mar_20180621':[3,5],
                'mar_20180727':[4,7],
                'mar_20180925':[5],
                'mar_20181113':[0],
                'mar_20181211':[0],
                'mar_20190205':[0,5],
                'mar_20190313':[4],
                'mar_20190516':[2,5]}

veg_dict = {'leo_20180606':[1,3,7,8],
            'leo_20180713':[1,5,9],
            'leo_20180920':[1,4,5],
            'leo_20190211':[4],
            'leo_20190328':[0,1,6],
            'leo_20190731':[3,7],
            'mar_20180601':[0,7],
            'mar_20180621':[1,7],
            'mar_20180727':[1,3],
            'mar_20180925':[1,3],
            'mar_20181113':[3],
            'mar_20181211':[2],
            'mar_20190205':[3],
            'mar_20190313':[1,5],
            'mar_20190516':[0]}

sand_dict = {'leo_20180606':[2,4,6],
             'leo_20180713':[2,6,8],
             'leo_20180920':[3],
             'leo_20190211':[3],
             'leo_20190328':[3],
             'leo_20190731':[4,5],
             'mar_20180601':[2,3],
             'mar_20180621':[0,2],
             'mar_20180727':[2,6,8],
             'mar_20180925':[0,4,2],
             'mar_20181113':[2,4],
             'mar_20181211':[3,1],
             'mar_20190205':[1,2,4],
             'mar_20190313':[0,2,3],
             'mar_20190516':[1,3,6]}

Now that we have each class dictionary, let's put all of those in another dictionary in order to specify the name of the class.
Like this:

In [ ]:
# JLL edit: added spaces for readability

l_dicts = {'no_sand': no_sand_dict,
           'sand': sand_dict,
           'water': water_dict,
           'veg':veg_dict}

Now, let's provide the paths to the geopackages (preferred, but also shapefiles are accepted) containing the label corrections, the watermasks and the shoremasks.

In [ ]:
# JLL edit: added spaces for readability, made paths mac compatible

label_corrections_path = Path(test_data_folder + r"/clean/label_corrections.gpkg")
watermasks_path        = Path(test_data_folder + r"/clean/watermasks.gpkg")
shoremasks_path        = Path(test_data_folder + r"/clean/shoremasks.gpkg")

All set!

Now we are ready to use the __.cleanit method__ to:

1. classify each point using the dictionaries provided
2. fine-tuning the label_k using the label correction polygons provided
3. eliminate water/swash points using the watermasks provided
4. clip to the shore area using the shoremasks provided


In [ ]:
P.cleanit(l_dicts=l_dicts,
          watermasks_path=watermasks_path,
          shoremasks_path=shoremasks_path,
          label_corrections_path=label_corrections_path)

DONE!

Now we have a cleaned and tidy classified dataset of elevation profiles, ready to be analysed to extract sediment specific altimetric, volumetric and behavioural dynamics information.

A new attribute has been stored, *ProfileSet.cleaning_steps*, which is a list of the cleaning steps applied to the current profiles.

In [ ]:
P.cleaning_steps

## Save it

To save the objects, just call the method *ProfileSet.save* with only two arguments:
1. the path to the folder where to save the data.
2. the name of the file you want to create.

In [ ]:
# JLL edit: changed the path to the test data folder, added and removed spaces for readability

dir_out = test_data_folder
name    = "test.p"
P.save(name, dir_out)
P

## Conclusion

In this notebook we extracted the elevation and colour information from all our rasters and used it with an iterative Silhouette analysis and KMeans algorithm to unsupervisingly partition the data into clusters of similar points. Then, we assigned meaning of these labels using an external GIS and cleaned and clipped the data into our final slassified dataset. In the next notebook we will finally obtain multi-scale information about sediment dynamics in each site/transect.

___